In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
deals = pd.read_excel('Deals_clean.xlsx')
calls = pd.read_excel('Calls_clean.xlsx')
spend = pd.read_excel('Spend_clean.xlsx')
contacts = pd.read_excel('Contacts_clean.xlsx')

In [ ]:
contacts = contacts.rename(columns={"id": "ContactID"})
calls = calls.rename(columns={"CONTACTID": "ContactID"})
deals = deals.rename(columns={"Contact Name": "ContactID"})

In [ ]:
print("Column names in 'deals' dataframe:")
print(deals.columns.tolist())

print("\nColumn names in 'spend' dataframe:")
print(spend.columns.tolist())

print("\nColumn names in 'contacts' dataframe:")
print(contacts.columns.tolist())

Column names in 'deals' dataframe:
['id', 'deal_owner_name', 'closing_date', 'quality', 'stage', 'lost_reason', 'page', 'campaign', 'content', 'term', 'source', 'payment_type', 'product', 'education_type', 'created_time', 'course_duration', 'months_of_study', 'initial_amount_paid', 'offer_total_amount', 'contact_name', 'city', 'level_of_deutsch_standardized', 'sla_seconds', 'longitude', 'latitude', 'country', 'federal_state']

Column names in 'spend' dataframe:
['date', 'source', 'campaign', 'impressions', 'spend', 'clicks', 'adgroup', 'ad']

Column names in 'contacts' dataframe:
['ContactID', 'contact_owner_name', 'created_time', 'modified_time']


In [ ]:
deals['product'].value_counts()

,count
product,
Digital Marketing,1942
UX/UI Design,999
Web Developer,564


### 1. Расчет юнит-экономики по продуктам

In [ ]:
def calculate_aov_and_revenue(deals: pd.DataFrame) -> pd.DataFrame:
    """
    Рассчитывает метрики AOV_i (Average Order Value per iteration)
    и Revenue_i (суммарную выручку) для каждой сделки.

    Args:
        deals (pd.DataFrame): Датафрейм со сделками. Должен содержать
            колонки 'months_of_study', 'offer_total_amount',
            'initial_amount_paid' и 'course_duration'.

    Returns:
        pd.DataFrame: Исходный датафрейм с добавленными колонками
            'AOV_i' и 'Revenue_i'.
    """
    deals = deals.copy()
    deals['AOV_i'] = deals.apply(
        lambda row: (
            (
                (row['months_of_study'] - 1)
                * (row['offer_total_amount'] - row['initial_amount_paid'])
            )
            / (row['course_duration'] - 1)
            + row['initial_amount_paid']
        ) / row['months_of_study']
        if row['months_of_study'] > 0 and row['course_duration'] > 1
        else 0,
        axis=1,
    )

    deals['Revenue_i'] = deals['AOV_i'] * deals['months_of_study']
    return deals


def calculate_product_unit_economics(
    deals: pd.DataFrame, contacts: pd.DataFrame, spend: pd.DataFrame
) -> pd.DataFrame:
    """
    Вычисляет метрики unit economics по каждому продукту.

    Args:
        deals (pd.DataFrame): Все сделки.
        contacts (pd.DataFrame): Датафрейм с контактами.
        spend (pd.DataFrame): Датафрейм с маркетинговыми расходами.

    Returns:
        pd.DataFrame: Таблица с метриками unit economics по продуктам.
    """
    won_deals = deals[deals['stage'] == 'Payment Done'].copy()

    # --- Метрики по продуктам ---
    product_won_deals = (
        won_deals.groupby('product')
        .agg({'id': 'nunique'})
        .reset_index()
        .rename(columns={'id': 'B'})
    )

    product_revenue = (
        won_deals.groupby('product')
        .agg({'Revenue_i': 'sum'})
        .reset_index()
        .rename(columns={'Revenue_i': 'REW'})
    )

    product_deals_count = (
        deals.groupby('product')
        .agg({'id': 'nunique'})
        .reset_index()
        .rename(columns={'id': 'T'})
    )

    # --- Объединение ---
    product_metrics = (
        product_deals_count.merge(product_won_deals, on='product', how='left')
        .merge(product_revenue, on='product', how='left')
        .fillna(0)
    )

    # --- Общие значения ---
    overall_UA = contacts['ContactID'].nunique()
    overall_AC = spend['spend'].sum()

    # --- Расчёты ---
    product_metrics['UA'] = overall_UA
    product_metrics['C1'] = (
        product_metrics['B'] / product_metrics['UA']
    ).replace([np.inf, -np.inf], np.nan).fillna(0)

    product_metrics['AC'] = overall_AC
    product_metrics['CPA'] = (
        product_metrics['AC'] / product_metrics['UA']
    ).replace([np.inf, -np.inf], np.nan).fillna(0)

    product_metrics['AOV'] = (
        product_metrics['REW'] / product_metrics['T']
    ).replace([np.inf, -np.inf], np.nan).fillna(0)

    product_metrics['APC'] = (
        product_metrics['T'] / product_metrics['B']
    ).replace([np.inf, -np.inf], np.nan).fillna(0)

    product_metrics['CLTV'] = (
        product_metrics['AOV'] * product_metrics['APC']
    ).replace([np.inf, -np.inf], np.nan).fillna(0)

    product_metrics['LTV'] = (
        product_metrics['CLTV'] * product_metrics['C1']
    ).replace([np.inf, -np.inf], np.nan).fillna(0)

    product_metrics['CM'] = (
        product_metrics['UA'] * (product_metrics['LTV'] - product_metrics['CPA'])
    ).replace([np.inf, -np.inf], np.nan).fillna(0)

    # --- Переупорядочиваем ---
    product_metrics = product_metrics[
        ['product', 'UA', 'B', 'C1', 'AC', 'CPA', 'REW',
         'T', 'AOV', 'APC', 'CLTV', 'LTV', 'CM']
    ]

    return product_metrics


def calculate_overall_metrics(
    deals: pd.DataFrame,
    contacts: pd.DataFrame,
    spend: pd.DataFrame,
    product_metrics: pd.DataFrame,
) -> dict:
    """
    Рассчитывает общие бизнес-метрики по всей компании.

    Args:
        deals (pd.DataFrame): Все сделки.
        contacts (pd.DataFrame): Контакты.
        spend (pd.DataFrame): Расходы.
        product_metrics (pd.DataFrame): Таблица метрик по продуктам.

    Returns:
        dict: Словарь с основными бизнес-показателями.
    """
    won_deals = deals[deals['stage'] == 'Payment Done']
    UA = contacts['ContactID'].nunique()
    B = won_deals.shape[0]
    C1 = B / UA if UA > 0 else 0
    AC = spend['spend'].sum()
    CPA = AC / UA if UA > 0 else 0
    T = deals['id'].nunique()
    total_revenue = product_metrics['REW'].sum()
    AOV = total_revenue / T if T > 0 else 0
    APC = T / B if B > 0 else 0
    CLTV = AOV * APC
    LTV = CLTV * C1
    CM = UA * (LTV - CPA)

    return {
        "UA": UA,
        "B": B,
        "C1": C1,
        "AC": AC,
        "CPA": CPA,
        "T": T,
        "AOV": AOV,
        "APC": APC,
        "CLTV": CLTV,
        "LTV": LTV,
        "CM": CM,
    }


def print_overall_metrics(metrics: dict) -> None:
    """
    Выводит бизнес-метрики в консоль в удобочитаемом формате.

    Args:
        metrics (dict): Словарь с метриками, рассчитанными функцией
        calculate_overall_metrics().
    """
    print("\n Общие бизнес-метрики:")
    print(f"Уникальные контакты (UA): {metrics['UA']}")
    print(f"Закрытые сделки (B): {metrics['B']}")
    print(f"Коэффициент конверсии (C1 = B/UA): {metrics['C1']:.4f}")
    print(f"Общие расходы (AC): {metrics['AC']:.2f}")
    print(f"Стоимость привлечения клиента (CPA = AC/UA): {metrics['CPA']:.2f}")
    print(f"Всего сделок (T): {metrics['T']}")
    print(f"Средний чек (AOV): {metrics['AOV']:.2f}")
    print(f"Средний цикл покупки (APC): {metrics['APC']:.2f}")
    print(f"Пожизненная ценность клиента (CLTV): {metrics['CLTV']:.2f}")
    print(f"LTV (CLTV * C1): {metrics['LTV']:.2f}")
    print(f"Маржинальная прибыль (CM = UA * (LTV - CPA)): {metrics['CM']:.2f}")


deals = calculate_aov_and_revenue(deals)

product_unit_economics_detailed = calculate_product_unit_economics(
    deals, contacts, spend
)

overall_metrics = calculate_overall_metrics(
    deals, contacts, spend, product_unit_economics_detailed
)

print_overall_metrics(overall_metrics)

display(product_unit_economics_detailed)


 Общие бизнес-метрики:
Уникальные контакты (UA): 18548
Закрытые сделки (B): 835
Коэффициент конверсии (C1 = B/UA): 0.0450
Общие расходы (AC): 134388.54
Стоимость привлечения клиента (CPA = AC/UA): 7.25
Всего сделок (T): 3505
Средний чек (AOV): 999.88
Средний цикл покупки (APC): 4.20
Пожизненная ценность клиента (CLTV): 4197.11
LTV (CLTV * C1): 188.95
Маржинальная прибыль (CM = UA * (LTV - CPA)): 3370196.46


,product,UA,B,C1,AC,CPA,REW,T,AOV,APC,CLTV,LTV,CM
0,Digital Marketing,18548,472,0.025447,134388.54,7.245446,2226240.0,1942,1146.364573,4.114407,4716.610169,120.025879,2091851.46
1,UX/UI Design,18548,227,0.012239,134388.54,7.245446,922565.0,999,923.488488,4.400881,4064.162996,49.739325,788176.46
2,Web Developer,18548,136,0.007332,134388.54,7.245446,355780.0,564,630.815603,4.147059,2616.029412,19.181583,221391.46


**Влияние увелитчения метрик юнит экономики  на 5% на СМ**

In [ ]:
def calculate_cm(row: pd.Series) -> float:
    """
    Calculates Contribution Margin (CM) based on the provided metrics.

    Args:
        row (pd.Series): Строка DataFrame с колонками 'UA', 'LTV', 'CPA'.

    Returns:
        float: Contribution Margin (CM).
    """
    ua = row['UA']
    ltv = row['LTV']
    cpa = row['CPA']
    cm = ua * (ltv - cpa)
    return cm


def calculate_cm_with_increase(
    row: pd.Series,
    metric_to_increase: str,
    percentage_increase: float = 0.05
) -> pd.Series:
    """
    Calculates CM after increasing or decreasing a specific metric
    by a given percentage. Handles all dependencies properly.

    Args:
        row (pd.Series): Строка DataFrame с метриками.
        metric_to_increase (str): Название метрики для изменения ('UA', 'C1', 'CPA', 'AOV', 'APC').
        percentage_increase (float, optional): Процент изменения (по умолчанию 5%).

    Returns:
        pd.Series: Обновлённая строка с пересчитанными значениями CM, LTV, CLTV и зависимыми метриками.
    """
    updated_row = row.copy()
    original_value = updated_row[metric_to_increase]

    # Увеличиваем все метрики кроме CPA (её уменьшаем)
    if metric_to_increase == 'CPA':
        updated_row[metric_to_increase] = original_value * (1 - percentage_increase)
    else:
        updated_row[metric_to_increase] = original_value * (1 + percentage_increase)

    # === Пересчёт зависимостей ===
    if metric_to_increase == 'UA':
        updated_row['C1'] = (
            updated_row['B'] / updated_row['UA']
            if updated_row['UA'] > 0 else 0
        )
        updated_row['CPA'] = (
            updated_row['AC'] / updated_row['UA']
            if updated_row['UA'] > 0 else 0
        )

    if metric_to_increase in ['AOV', 'APC']:
        updated_row['CLTV'] = updated_row['AOV'] * updated_row['APC']
    else:
        updated_row['CLTV'] = row['AOV'] * row['APC']

    # Пересчёт LTV и CM
    updated_row['LTV'] = updated_row['CLTV'] * updated_row['C1']
    updated_row['CM'] = calculate_cm(updated_row)

    return updated_row


# === Основные расчёты ===
if 'Baseline CM' not in product_unit_economics_detailed.columns:
    product_unit_economics_detailed['Baseline CM'] = (
        product_unit_economics_detailed.apply(calculate_cm, axis=1)
    )

metrics_to_analyze = ['UA', 'C1', 'CPA', 'AOV', 'APC']
results_list = []

for _, base_row in product_unit_economics_detailed.iterrows():
    product_name = base_row['product']
    baseline_cm = base_row['Baseline CM']

    # Базовый сценарий
    baseline_scenario = base_row.copy()
    baseline_scenario['Сценарий'] = f'Базовый ({product_name})'
    results_list.append(baseline_scenario)

    # Сценарии изменений
    for metric in metrics_to_analyze:
        scenario_row = calculate_cm_with_increase(base_row, metric, percentage_increase=0.05)
        scenario_row['Сценарий'] = (
            f"{metric} {'-5%' if metric == 'CPA' else '+5%'} ({product_name})"
        )
        results_list.append(scenario_row)

# === Сборка результатов ===
df_results_detailed = pd.DataFrame(results_list)
final_columns = [
    'Сценарий', 'UA', 'B', 'C1', 'AC', 'CPA', 'REW', 'T',
    'AOV', 'APC', 'CLTV', 'LTV', 'CM'
]
df_results_detailed = df_results_detailed[
    [c for c in final_columns if c in df_results_detailed.columns]
]

# === Визуализация ===
print("\n Влияние изменения метрик на CM по продуктам (±5%):")

expected_products = ['Digital Marketing', 'UX/UI Design', 'Web Developer']

for product in expected_products:
    df_prod = df_results_detailed[
        df_results_detailed['Сценарий'].str.contains(f'({product})', regex=False)
    ].copy()

    df_prod = df_prod.loc[:, ~df_prod.columns.duplicated()].reset_index(drop=True)
    df_prod['Сценарий'] = df_prod['Сценарий'].str.replace(f' ({product})', '', regex=False)

    baseline_row = df_prod[df_prod['Сценарий'].str.contains('Базовый')].iloc[0]
    max_cm = df_prod['CM'].max()

    # === Подсветка ===
    def highlight_cells(val: float, col_name: str) -> str:
        """Определяет стиль ячейки в зависимости от изменения метрики."""
        try:
            base_val = float(baseline_row[col_name])
            val_f = float(val)

            # Подсветка всех метрик выше базового (для CPA — ниже базового)
            if col_name != 'CM':
                if (col_name == 'CPA' and val_f < base_val - 0.0001) or (
                    col_name != 'CPA' and val_f > base_val + 0.0001
                ):
                    return (
                        'background-color: #E8F8F5; color: #1E8449; font-weight: bold;'
                    )

            # Подсветка только максимального CM
            if col_name == 'CM' and abs(val_f - max_cm) < 0.001:
                return (
                    'background-color: #D5F5E3; color: #145A32; font-weight: bold;'
                )
        except Exception:
            pass
        return ''

    cols_existing = [
        c for c in ['UA', 'C1', 'CPA', 'AOV', 'APC', 'CLTV', 'LTV', 'CM']
        if c in df_prod.columns
    ]

    styled_df = (
        df_prod.style
        .format({
            'UA': '{:,.2f}', 'B': '{:,.2f}', 'C1': '{:.4f}', 'AC': '{:,.2f}',
            'CPA': '{:,.2f}', 'REW': '{:,.2f}', 'T': '{:,.2f}',
            'AOV': '{:,.2f}', 'APC': '{:,.2f}', 'CLTV': '{:,.2f}',
            'LTV': '{:,.2f}', 'CM': '{:,.2f}',
        })
        .apply(lambda col: [highlight_cells(v, col.name) for v in col], subset=cols_existing)
        .set_caption(f"Влияние изменения метрик юнит-экономики на CM: {product} (±5%)")
        .set_table_styles([
            {
                'selector': 'caption',
                'props': [
                    ('color', '#2E4053'),
                    ('font-size', '15px'),
                    ('font-weight', 'bold'),
                ],
            },
            {
                'selector': 'th',
                'props': [
                    ('background-color', '#D6EAF8'),
                    ('font-weight', 'bold'),
                ],
            },
            {'selector': 'td', 'props': [('border', '1px solid #AEB6BF')]},
        ])
    )

    display(styled_df)


 Влияние изменения метрик на CM по продуктам (±5%):


,Сценарий,UA,B,C1,AC,CPA,REW,T,AOV,APC,CLTV,LTV,CM
0,Базовый,"18,548.00",472.00,0.0254,"134,388.54",7.25,"2,226,240.00","1,942.00","1,146.36",4.11,"4,716.61",120.03,"2,091,851.46"
1,UA +5%,"19,475.40",472.00,0.0242,"134,388.54",6.90,"2,226,240.00","1,942.00","1,146.36",4.11,"4,716.61",114.31,"2,091,851.46"
2,C1 +5%,"18,548.00",472.00,0.0267,"134,388.54",7.25,"2,226,240.00","1,942.00","1,146.36",4.11,"4,716.61",126.03,"2,203,163.46"
3,CPA -5%,"18,548.00",472.00,0.0254,"134,388.54",6.88,"2,226,240.00","1,942.00","1,146.36",4.11,"4,716.61",120.03,"2,098,570.89"
4,AOV +5%,"18,548.00",472.00,0.0254,"134,388.54",7.25,"2,226,240.00","1,942.00","1,203.68",4.11,"4,952.44",126.03,"2,203,163.46"
5,APC +5%,"18,548.00",472.00,0.0254,"134,388.54",7.25,"2,226,240.00","1,942.00","1,146.36",4.32,"4,952.44",126.03,"2,203,163.46"


,Сценарий,UA,B,C1,AC,CPA,REW,T,AOV,APC,CLTV,LTV,CM
0,Базовый,"18,548.00",227.00,0.0122,"134,388.54",7.25,"922,565.00",999.00,923.49,4.40,"4,064.16",49.74,"788,176.46"
1,UA +5%,"19,475.40",227.00,0.0117,"134,388.54",6.90,"922,565.00",999.00,923.49,4.40,"4,064.16",47.37,"788,176.46"
2,C1 +5%,"18,548.00",227.00,0.0129,"134,388.54",7.25,"922,565.00",999.00,923.49,4.40,"4,064.16",52.23,"834,304.71"
3,CPA -5%,"18,548.00",227.00,0.0122,"134,388.54",6.88,"922,565.00",999.00,923.49,4.40,"4,064.16",49.74,"794,895.89"
4,AOV +5%,"18,548.00",227.00,0.0122,"134,388.54",7.25,"922,565.00",999.00,969.66,4.40,"4,267.37",52.23,"834,304.71"
5,APC +5%,"18,548.00",227.00,0.0122,"134,388.54",7.25,"922,565.00",999.00,923.49,4.62,"4,267.37",52.23,"834,304.71"


,Сценарий,UA,B,C1,AC,CPA,REW,T,AOV,APC,CLTV,LTV,CM
0,Базовый,"18,548.00",136.00,0.0073,"134,388.54",7.25,"355,780.00",564.00,630.82,4.15,"2,616.03",19.18,"221,391.46"
1,UA +5%,"19,475.40",136.00,0.0070,"134,388.54",6.90,"355,780.00",564.00,630.82,4.15,"2,616.03",18.27,"221,391.46"
2,C1 +5%,"18,548.00",136.00,0.0077,"134,388.54",7.25,"355,780.00",564.00,630.82,4.15,"2,616.03",20.14,"239,180.46"
3,CPA -5%,"18,548.00",136.00,0.0073,"134,388.54",6.88,"355,780.00",564.00,630.82,4.15,"2,616.03",19.18,"228,110.89"
4,AOV +5%,"18,548.00",136.00,0.0073,"134,388.54",7.25,"355,780.00",564.00,662.36,4.15,"2,746.83",20.14,"239,180.46"
5,APC +5%,"18,548.00",136.00,0.0073,"134,388.54",7.25,"355,780.00",564.00,630.82,4.35,"2,746.83",20.14,"239,180.46"


Таким образом, метриками, которые оказывают наибольшее положительное влияние на Contribution Margin при их увеличении на 5%, являются Conversion Rate (C1), Average Order Value (AOV) и Average Purchase Cycle (APC). Увеличение любой из этих метрик на 5% приводит к более значительному росту CM в процентном выражении по сравнению с таким же процентным увеличением UA.

Следовательно, точки роста для бизнеса, на которые стоит сфокусироваться для увеличения CM, лежат в области повышения конверсии, среднего чека и частоты покупок (Average Purchase Cycle). Эти метрики имеют наибольший рычаг воздействия на рентабельность бизнеса.

# **Гипотеза для A/B теста 1.**

**Гипотеза:**
Если внедрить оптимизированный подход к работе с лидами через дополнительное обучение менеджеров по всем продуктам ("Digital Marketing", "UX/UI Design", "Web Developer"), то общий коэффициент конверсии (доля сделок со стадией "Payment Done" от числа лидов из этих источников) увеличится минимум на 5 процентных пунктов в течение 2 недель.

**Нулевая гипотеза:**
Изменение процесса работы с лидами не повлияет на общий коэффициент конверсии.

**Контрольная группа (A):**
Работает по стандартному процессу обработки лидов.

**Тестовая группа (B):**
Использует оптимизированный подход (дополнительное обучение/стандартизация).

Распределение новых лидов — случайное и равномерное между группами.

**Условия A/B теста:**

Рандомизация лидов 50/50 между группами.

Продолжительность эксперимента 2 недели.

**Основная метрика:** прирост общего коэффициента конверсии в тестовой группе относительно контрольной.

**Критерии успеха:**

Конверсия должна увеличиться минимум на 5 процентных пунктов  в тестовой группе.

Достижение статистически значимого улучшения относительно контрольной группы.

In [ ]:
contacts['created_time'] = pd.to_datetime(contacts['created_time'], errors='coerce')

min_contact_date = contacts['created_time'].min()
max_contact_date = contacts['created_time'].max()
contact_time_range_days = (
    (max_contact_date - min_contact_date).days
    if pd.notna(min_contact_date) and pd.notna(max_contact_date)
    else 1
)

overall_UA = contacts['ContactID'].nunique()
overall_avg_daily_ua = (
    overall_UA / contact_time_range_days if contact_time_range_days > 0 else 0
)

print(f"Общее количество уникальных контактов (UA): {overall_UA}")
print(f"Период данных по контактам (в днях): {contact_time_range_days}")
print(f"Среднее количество уникальных контактов (UA) за день: {overall_avg_daily_ua:.2f}")

Общее количество уникальных контактов (UA): 18548
Период данных по контактам (в днях): 360
Среднее количество уникальных контактов (UA) за день: 51.52


In [ ]:
deals_renamed = deals.rename(columns={"contact_name": "ContactID"})
contacts['ContactID'] = contacts['ContactID'].astype(str)
deals_renamed['ContactID'] = deals_renamed['ContactID'].astype(str)

contacts_with_deals = pd.merge(
    contacts,
    deals_renamed[['ContactID', 'product']],
    on='ContactID',
    how='left'
)

contacts_per_product = (
    contacts_with_deals.dropna(subset=['product'])
    .groupby('product')['ContactID']
    .nunique()
    .reset_index()
    .rename(columns={'ContactID': 'Contacts'})
)

total_ua_with_product = contacts_per_product['Contacts'].sum()
contacts_per_product['Contact Share'] = (
    contacts_per_product['Contacts'] / total_ua_with_product
    if total_ua_with_product > 0 else 0
)

contacts_per_product['Avg Daily UA'] = (
    contacts_per_product['Contact Share'] * overall_avg_daily_ua
)

product_ab_test_params_ua = pd.merge(
    product_unit_economics_detailed[['product', 'C1']],
    contacts_per_product[['product', 'Avg Daily UA']],
    on='product',
    how='left'
)

print("Параметры A/B теста по продуктам (на основе UA):")
display(product_ab_test_params_ua)

Параметры A/B теста по продуктам (на основе UA):


,product,C1,Avg Daily UA
0,Digital Marketing,0.025447,28.552147
1,UX/UI Design,0.012239,14.671595
2,Web Developer,0.007332,8.298481


In [ ]:
def calculate_ab_params(baseline_c1, avg_daily_ua, effect_size=0.05):
    """Вычисление параметров A/B теста для продукта"""
    n_per_group = (16 * baseline_c1 * (1 - baseline_c1)) / (effect_size ** 2)
    ua_per_group_per_day = avg_daily_ua / 2 if avg_daily_ua > 0 else 0
    days_to_recruit = (
        2 * n_per_group / ua_per_group_per_day
        if ua_per_group_per_day > 0 else np.inf
    )
    return n_per_group, days_to_recruit

In [ ]:
print("\n=== Гипотеза 1: Дополнительное обучение менеджеров (все продукты) ===")

X = 0.05  # 5 п.п.
results_h1 = {}

for _, row in product_ab_test_params_ua.iterrows():
    product = row['product']
    baseline_c1 = row['C1']
    avg_daily_ua = row['Avg Daily UA']

    n, days = calculate_ab_params(baseline_c1, avg_daily_ua, X)
    results_h1[product] = {'Размер выборки (на группу)': n, 'Дней для набора': days}

for product, res in results_h1.items():
    print(f"\n--- {product} ---")
    print(f"Необходимый размер выборки на группу (для X={X*100:.1f}%): {res['Размер выборки (на группу)']:.2f}")
    print(f"Примерное количество дней для набора выборки: {res['Дней для набора']:.2f}")


=== Гипотеза 1: Дополнительное обучение менеджеров (все продукты) ===

--- Digital Marketing ---
Необходимый размер выборки на группу (для X=5.0%): 158.72
Примерное количество дней для набора выборки: 22.24

--- UX/UI Design ---
Необходимый размер выборки на группу (для X=5.0%): 77.37
Примерное количество дней для набора выборки: 21.09

--- Web Developer ---
Необходимый размер выборки на группу (для X=5.0%): 46.58
Примерное количество дней для набора выборки: 22.45


**Общий вывод:**

Для каждого продукта, чтобы обнаружить увеличение конверсии на 5 процентных пунктов с достаточной статистической мощностью, потребуется набор выборки, который, согласно расчетам (примерно 21-23 дня), превышает запланированный 2-недельный срок эксперимента.

**Вывод: Для проведения A/B теста по первой гипотезе в рамках 2-недельного срока с ожидаемым эффектом в 5 п.п., вероятно, потребуется увеличить количество лидов или пересмотреть длительность эксперимента.**

# **Гипотеза для A/B теста 2.**

**Н. Гипотеза:**
Если внести точечные изменения в процесс работы с новыми лидами по продукту "Digital Marketing" а именно оптимизировать скрипт первого звонка менеджера, то коэффициент конверсии (доля сделок со стадией "Payment Done" от числа лидов по этому продукту) увеличится минимум на 7 процентный пункт в течение 2 недель.

**Нулевая гипотеза:**
Изменение скрипта первого звонка в процесс работы с новыми лидами по продукту "Digital Marketing" не повлияет на коэффициент конверсии для этого продукта.

**Контрольная группа (A):**
Новые лиды по продукту "Digital Marketing", обрабатываемые по стандартному процессу.

**Тестовая группа (B):**
Новые лиды по продукту "Digital Marketing", обрабатываемые с учетом точечных оптимизаций процесса.

**Распределение новых лидов "Digital Marketing"** — случайное и равномерное между контрольной и тестовой группами.

**Условия A/B теста:**

*   Рандомизация новых лидов "Digital Marketing" 50/50 между группами.
*   Продолжительность эксперимента 2 недели.
*   Основная метрика: прирост коэффициента конверсии в тестовой группе относительно контрольной для продукта "Digital Marketing".

**Критерии успеха:**

*   Конверсия по продукту "Digital Marketing" должна увеличиться минимум на 7 процентный пункт в тестовой группе.
*   Достижение статистически значимого улучшения относительно контрольной группы в течение 2 недель.

In [ ]:
print("\n=== Гипотеза 2: Оптимизация скрипта первого звонка (только Digital Marketing) ===")

# Можно задать другой эффект (например, 3 процентных пункта)
X_h2 = 0.07  # <= тут просто меняешь значение

dm_row = product_ab_test_params_ua.loc[
    product_ab_test_params_ua['product'] == 'Digital Marketing'
].iloc[0]

baseline_c1_dm = dm_row['C1']
avg_daily_ua_dm = dm_row['Avg Daily UA']

n_dm, days_dm = calculate_ab_params(baseline_c1_dm, avg_daily_ua_dm, X_h2)

print(f"\n--- Digital Marketing ---")
print(f"Базовый коэффициент конверсии (C1): {baseline_c1_dm:.4f}")
print(f"Необходимый размер выборки на группу (для X={X_h2*100:.1f}%): {n_dm:.2f}")
print(f"Примерное количество дней для набора выборки: {days_dm:.2f}")


=== Гипотеза 2: Оптимизация скрипта первого звонка (только Digital Marketing) ===

--- Digital Marketing ---
Базовый коэффициент конверсии (C1): 0.0254
Необходимый размер выборки на группу (для X=7.0%): 80.98
Примерное количество дней для набора выборки: 11.34


Вывод по расчету параметров A/B теста

Исходя из  расчетов для Гипотезы 2 (Оптимизация скрипта первого звонка для Digital Marketing с ожидаемым увеличением конверсии на 7 процентных пунктов):

Необходимый размер выборки на группу составляет 80.98 уникальных контактов.
Примерное количество дней для набора этой выборки составляет 11.34 дней.
Поскольку запланированная длительность эксперимента — 2 недели (14 дней), а расчетное время набора выборки (11.34 дней) меньше 14 дней, то тест по продукту "Digital Marketing" с ожидаемым эффектом в 7% укладывается в 2-недельный срок.

**Вывод: Параметры A/B теста для Гипотезы 2 позволяют провести эксперимент в течение 2 недель и получить статистически значимые результаты, если ожидаемый эффект (увеличение C1 на 7 п.п.) будет достигнут.**

# Дерево Метрик

**1. Целевой показатель:**

*   **CM** (Contribution Margin)

**2. Финансовые метрики:**

*   **REW** (Revenue)

**3. Метрики юнит-экономики:**

*   **UA** (Уникальные контакты)
*   **C1** (Коэффициент конверсии)
*   **AOV** (Average Order Value / Средний чек)
*   **APC** (Average Purchase Cycle / Средний цикл покупки)
*   **CPA** (Cost Per Acquisition / Стоимость привлечения клиента)

**4. Продуктовые метрики:**

*   **AC** (Общие расходы)
*   **T** (Всего сделок)
*   **REW_i** (Выручка по сделке )
*   **B** (Закрытые сделки)
*   **LTV** (Lifetime Value / Пожизненная ценность клиента)
*   **CLTV** (Customer Lifetime Value / Пожизненная ценность клиента)
*   **AOV_i** (Average Order Value per iteration / Средний чек за итерацию - используется для расчета Revenue_i)

**5. Атомные метрики:**

*   **ID** (Идентификатор сделки/контакта)
*   **Page**
*   **Content**
*   **Term**
*   **Source**
*   **Created Time** (Время создания)
*   **Payment Type** (Тип оплаты)
*   **Product** (Продукт)
*   **Education Type** (Тип образования)
*   **City** (Город)
*   **Course duration** (Длительность курса)
*   **Months of study** (Месяцы обучения)
*   **Initial Amount Paid** (Первоначальная оплата)